In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, OneHotEncoder, PolynomialFeatures, LabelEncoder
from sklearn.tree import DecisionTreeClassifier

from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score

from sklearn.model_selection import train_test_split

from imblearn.over_sampling import RandomOverSampler  # балансировка классов
from collections import Counter

import seaborn as sns

### Начало!!

In [2]:
#df = pd.read_csv('post_07_10_2020_1.csv', sep=';')
df = pd.read_csv('post_18_12_2020.csv', sep=';')

In [3]:
df

,N,Артикул,Код характеристики,Группа номенклатуры,Основной поставщик,Ед. изм.,Фасовка айрис,Коэффициент единицы,Средние продажи,Цена продажи,...,Кол-во дискретов в ктр были продажи,Дней пристутств. продаже,Закупка за период,Статус товара Айриса,Категория товара ABC,Категория товара АВС Айриса,Закупочная цена,Коэффициент минимальной поставки излишков,Минимальный остаток,Нормативный остаток
0,1,699065,NaN,Силуэты разные,ВАСИЛИСА ООО,шт,5.0,1.0,0.781,64.0,...,4.0,374,22.0,Удалить,B,C,17.96,NaN,1.0,5.0
1,2,680157,NaN,Сердца,BOVELACCI S.R.L. (Италия),шт,10.0,1.0,1.653,63.0,...,7.0,374,147.0,Регулярный,B,A,10.50,NaN,2.0,10.0
2,3,695153,NaN,Сердца,BOVELACCI S.R.L. (Италия),шт,10.0,1.0,0.773,29.0,...,6.0,374,201.0,Регулярный,B,C,7.31,NaN,2.0,10.0
3,4,680158,NaN,Сердца,BOVELACCI S.R.L. (Италия),шт,5.0,1.0,2.180,101.0,...,5.0,374,236.0,Регулярный,A,A,51.60,NaN,1.0,5.0
4,5,680160,NaN,Яйца,BOVELACCI S.R.L. (Италия),шт,10.0,1.0,2.317,28.0,...,6.0,311,887.0,Регулярный,A,A,12.83,NaN,3.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8017,8 018,901404,NaN,Мебель,ВАСИЛИСА ООО,шт,1.0,1.0,0.342,173.0,...,4.0,364,33.0,Удалить,B,C,173.00,NaN,0.0,2.0
8018,8 019,903487,NaN,Основы для часов,ВАСИЛИСА ООО,шт,1.0,1.0,0.432,205.0,...,4.0,186,117.0,Регулярный,A,B,67.20,NaN,NaN,NaN
8019,8 020,558426,NaN,Пасхальные заготовки,ВАСИЛИСА ООО,шт,1.0,1.0,1.587,64.0,...,2.0,51,124.0,Ждет решения,B,C,21.00,NaN,NaN,NaN
8020,8 021,558425,NaN,Пасхальные заготовки,ВАСИЛИСА ООО,шт,1.0,1.0,1.587,64.0,...,2.0,51,142.0,Ждет решения,A,C,21.00,NaN,NaN,NaN


In [ ]:
df[df['Минимальный остаток']==90] # быстрая оценка значений классов:

In [ ]:
df[df['Закупка за период']<0] #быстрая оценка признаков

In [ ]:
df.info()

#### Начинаем работу со строками:

In [ ]:
df.drop(['N', 'Артикул', 'Код характеристики', 'Коэффициент минимальной поставки излишков'], axis=1, inplace=True)

In [ ]:
df.dropna(subset=['Статус товара Айриса', 'Ед. изм.', 'Фасовка айрис', 
                  'Коэффициент единицы', 'Цена продажи', 'Закупочная цена'], inplace=True)

In [ ]:
ind_for_drop = df[(df['Средние продажи'].isna() & df['Кол-во сделок'] > 0) | (df['Закупка за период'] < 0)].index #### удалить строчки, в ктр средние продажи = 0, а кол-во сделок> 0
df_1 = df.drop(list(ind_for_drop))

In [ ]:
list(ind_for_drop)

In [ ]:
df_1.describe(include='object')

In [ ]:
df_1[['Минимальный остаток', 'Нормативный остаток']]

In [ ]:
# Поиграем с типами данных для targets
# df[['Минимальный остаток', 'Нормативный остаток']] = df[['Минимальный остаток', 'Нормативный остаток']].round()


In [ ]:
df_1[['Минимальный остаток', 'Нормативный остаток']]

In [ ]:
df_1.describe()

In [ ]:
df_1.info()

In [ ]:
X = df_1.drop(['Минимальный остаток', 'Нормативный остаток'], axis=1)
y = df_1.loc[:, ['Минимальный остаток', 'Нормативный остаток']]   # таргет

In [ ]:
y_need = y[df_1['Минимальный остаток'].isna()] # отфильтровываем неизвестные значения МИН
X_need = X.loc[list(y_need.index)] 

y_filter = y.drop(list(y_need.index))      # формируем значения МИН и МАХа на ктр будем обучаться

X_filter = X.drop(list(y_need.index))      # формируем соответствующие данные на ктр будем обучаться

In [ ]:
y_filter_min = y_filter['Минимальный остаток']
y_filter_max = y_filter['Нормативный остаток']

In [ ]:
y_filter_min.shape


In [ ]:
X_filter


In [ ]:
y_filter

#### Построим гистограммы, чтоб визуально посмотреть на частотное распределение меток классов МИН и МАХ: 

In [ ]:
from matplotlib.pyplot import figure
figure(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
sns.countplot(x='Минимальный остаток', data=y_filter)
plt.xlabel('Мин. остаток', fontsize=12)
plt.ylabel('Count', fontsize=13)

In [ ]:
print(y_filter.loc[:, 'Минимальный остаток'].value_counts())
len(y_filter.loc[:, 'Минимальный остаток'].value_counts())

In [ ]:
# figure(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
fig,ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
sns.countplot(x='Нормативный остаток', data=y_filter)
# plt.axis([0.0, 70])
plt.xlabel('Норм. остаток', fontsize=12)
plt.ylabel('Count', fontsize=13)
#ax.set(xlim=(0, 300))
plt.axis([0.0, 25, 0, 1800])
#ax.set_xticklabels(range(0,50))
plt.show()


# figure(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
fig,ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
sns.countplot(x='Нормативный остаток', data=y_filter)
# plt.axis([0.0, 70])
plt.xlabel('Норм. остаток', fontsize=12)
plt.ylabel('Count', fontsize=13)
#plt.xticks(y_filter.loc[:, 'Нормативный остаток'])
#ax.set(xlim=(0, 100))
plt.axis([26, 50, 0, 1800])
# ax.set_xticklabels(range(0,50))
plt.show()


In [ ]:
print(y_filter.loc[:, 'Нормативный остаток'].value_counts())
len(y_filter.loc[:, 'Нормативный остаток'].value_counts())

### Сделаем балансироваку классов:

In [ ]:
le = LabelEncoder()

y_filter_min_le = le.fit_transform(y_filter_min)
y_filter_max_le = le.fit_transform(y_filter_max)

In [ ]:
ros = RandomOverSampler(random_state=0)

X_resampled_min, y_resampled_min = ros.fit_resample(X_filter, y_filter_min_le)
X_resampled_max, y_resampled_max = ros.fit_resample(X_filter, y_filter_max_le)


In [ ]:
print(len(X_resampled_min))
print(len(X_resampled_max))

In [ ]:
pd.DataFrame(X_resampled_max)
#pd.DataFrame(y_resampled_max)

In [ ]:
le.inverse_transform(y_resampled_max)

In [ ]:
pd.DataFrame(le.inverse_transform(y_resampled_max))

In [ ]:
print(sorted(Counter(y_resampled_min).items()))
print(sorted(Counter(y_resampled_max).items()))

In [ ]:
# классы сбалансированы, так что (stratify=y_filter) делать не будем
X_train_min, X_test_min, y_train_min, y_test_min = train_test_split(
    X_resampled_min, y_resampled_min, test_size=0.33, random_state=42
)
X_train_max, X_test_max, y_train_max, y_test_max = train_test_split(
    X_resampled_max, y_resampled_max, test_size=0.33, random_state=42
)

In [ ]:
# Начнем трансформировать колонки, в зависимости от типа значений:
categorical_features_min = X_train_min.select_dtypes(include=[object])
categorical_features_max = X_train_max.select_dtypes(include=[object])

In [ ]:
categorical_features_max

In [ ]:
numerical_features_min = X_train_min.select_dtypes(include=[np.number])
numerical_features_max = X_train_max.select_dtypes(include=[np.number])

In [ ]:
numerical_features_max

In [ ]:
y_train_max

In [ ]:
X_train_min.info()

In [ ]:
# будем вставлять 0
numerical_features_min_1 = numerical_features_min[
    ['Средние продажи', 'Продано всего','Кол-во сделок', 'Кол-во дискретов в ктр были продажи', 'Закупка за период']
]         
numerical_features_min_2 = numerical_features_min.drop(numerical_features_min_1.columns, axis=1)

numerical_features_max_1 = numerical_features_max[
    ['Средние продажи', 'Продано всего','Кол-во сделок', 'Кол-во дискретов в ктр были продажи', 'Закупка за период']
]         
numerical_features_max_2 = numerical_features_max.drop(numerical_features_max_1.columns, axis=1)

In [ ]:
# здесь устанавливаем, что заполняем нулями отсутствующ. значения
imp_const = SimpleImputer(missing_values=np.nan, strategy='constant')

In [ ]:
# указываем колонки для трансформации (общие и для мин и для мах):
numerical_features_1_col = numerical_features_min_1.columns  # будем вставлять 0 и производить PolynomialFeatures
numerical_features_2_col = numerical_features_min_2.columns  # будем только производить PolynomialFeatures
categorical_features_col = categorical_features_min.columns  # будем производить OHE

# указываем какой препроцессинг будем делать на разных колонках (параметры скалирования зададим непосредств.в grid_search):
numerical_transformer_1 = Pipeline(
    steps=[
        ('imp_const', imp_const),
        ('poly', PolynomialFeatures(interaction_only=True,include_bias = False)),
        ('scaler', 'passthrough')
        ]
)
numerical_transformer_2 = Pipeline(
    steps=[
        ('poly', PolynomialFeatures(interaction_only=True,include_bias = False)),
        ('scaler', 'passthrough')
        ]
)
categorical_transformer = Pipeline(
    steps=[
        ('imp_const', imp_const),
        ('ОНЕ', OneHotEncoder(sparse=False, handle_unknown='ignore'))
        ]
)

# создаем общий трансформатор:
data_transformer = ColumnTransformer(
    transformers=[
        ('numerical_1', numerical_transformer_1, numerical_features_1_col),
        ('numerical_2', numerical_transformer_2, numerical_features_2_col),
        ('categorical', categorical_transformer, categorical_features_col)])

### Наконец то подошли к классификации!!

In [ ]:
# 1. обратить внимание на балансировку внутри эстиматора....нужна ли она?
# 2. стандартизация нужна ли для деревьев и леса...и какая? (вроде для деревьев не нужна стандартизация)
# 3. Сейчас включен рандом форест на 5ой версии))

In [ ]:
# DecisionTreeClassifier
en_Tree = Pipeline(
    steps=[('preprocessor', data_transformer),
        ('Tree', DecisionTreeClassifier(random_state=42, class_weight='balanced'))
        ]
)

# RandomForestClassifier
en_RF = Pipeline(
    steps=[('preprocessor', data_transformer),
        ('RF', RandomForestClassifier(random_state=42, class_weight='balanced'))
        ]
)

In [ ]:
# Создаем сетку поиска

# для DecisionTreeClassifier:
param_grid_tree = {
    'preprocessor__numerical_1__scaler': [StandardScaler()],
    'Tree__criterion':['gini','entropy'],
    'Tree__max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]
}

# для RandomForestClassifier:
param_grid_rf = {
    'preprocessor__numerical_1__scaler': [StandardScaler()],
    'RF__criterion':['gini','entropy'],
    'RF__max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]
}


In [ ]:
grid_search_min = GridSearchCV(en_RF, param_grid=param_grid_rf)
grid_search_max = GridSearchCV(en_RF, param_grid=param_grid_rf)

In [ ]:
en_RF.get_params().keys()

In [ ]:
grid_min = grid_search_min.fit(X_train_min, y_train_min)

In [ ]:
grid_min.best_params_

In [ ]:
grid_max = grid_search_max.fit(X_train_max, y_train_max)

In [ ]:
grid_max.best_params_

In [ ]:
grid_min.score(X_train_min, y_train_min)

In [ ]:
grid_max.score(X_train_max, y_train_max)

In [ ]:
pred_min = grid_min.predict(X_test_min)
pred_max = grid_max.predict(X_test_max)

In [ ]:
# С этого момента уже не сохранилось!!! 28.12.2020
# НЕ ЗАБЫТЬ СДЕЛАТЬ ОБРАТНОЕ ПРЕОБРАЗОВАН на  ЛЭЙБЛ!!!!(обратное рпеобразование делать после всех метрик, 
# иначе будет выдавать ошибку: continuous is not supported)

In [ ]:
frame_min = [
    pd.DataFrame(pred_min, columns=['predict']), 
    pd.DataFrame(y_test_min, columns=['Минимальный остаток']).reset_index(drop=True), X_test_min['Фасовка айрис'].reset_index(drop=True)
]
frame_max = [
    pd.DataFrame(pred_max, columns=['predict']), 
    pd.DataFrame(y_test_max, columns=['Нормативный остаток']).reset_index(drop=True), X_test_max['Фасовка айрис'].reset_index(drop=True)
]


In [ ]:
df3_min = pd.concat(frame_min, axis=1)
df3_max = pd.concat(frame_max, axis=1)

In [ ]:
df3_min['prophet'] = df3_min['predict']    # 'это только для МИНИМАЛЬНОГО остатка !!!'
# df3_max['prophet'] = df3_max[['predict', 'Фасовка айрис']].max(axis=1)    # 'это только для НОРМАТИВНОГО остатка !!!'
df3_max['prophet'] = df3_max['predict']

In [ ]:
IPython.display.HTML(df3_max.to_html())

#### Посмотрим на метрики:

In [ ]:
accuracy_score(df3_min['Минимальный остаток'], df3_min['prophet'])

In [ ]:
accuracy_score(df3_max['Нормативный остаток'], df3_max['prophet'])

In [ ]:
precision_min, recall_min, fscore_min, support_min = score(df3_min['Минимальный остаток'], df3_min["prophet"])
precision_macro_min, recall_macro_min, fscore_macro_min, support_macro_min = score(df3_min['Минимальный остаток'], df3_min["prophet"], average='macro')
precision_micro_min, recall_micro_min, fscore_micro_min, support_micro_min = score(df3_min['Минимальный остаток'], df3_min["prophet"], average='micro')



In [ ]:
precision_max, recall_max, fscore_max, support_max = score(df3_max['Нормативный остаток'], df3_max["prophet"])
precision_macro_max, recall_macro_max, fscore_macro_max, support_macro_max = score(df3_max['Нормативный остаток'], df3_max["prophet"], average='macro')
precision_micro_max, recall_micro_max, fscore_micro_max, support_micro_max = score(df3_max['Нормативный остаток'], df3_max["prophet"], average='micro')

In [ ]:
score_precision_min = pd.DataFrame((precision_min.reshape(26, 1)), columns=['precision_min'])
score_recall_min = pd.DataFrame((recall_min.reshape(26, 1)), columns=['recall_min'])
score_fscore_min = pd.DataFrame((fscore_min.reshape(26, 1)), columns=['fscore_min'])
score_support_min = pd.DataFrame((support_min.reshape(26, 1)), columns=['support_min'])
for_frame_min = [score_precision_min, score_recall_min, score_fscore_min, score_support_min] 
score_all_min = pd.concat(for_frame_min, axis=1)
score_all_min

In [ ]:
score_precision_max = pd.DataFrame((precision_max.reshape(50, 1)), columns=['precision_max'])
score_recall_max = pd.DataFrame((recall_max.reshape(50, 1)), columns=['recall_max'])
score_fscore_max = pd.DataFrame((fscore_max.reshape(50, 1)), columns=['fscore_max'])
score_support_max = pd.DataFrame((support_max.reshape(50, 1)), columns=['support_max'])
for_frame_max = [score_precision_max, score_recall_max, score_fscore_max, score_support_max] 
score_all_max = pd.concat(for_frame_max, axis=1)
score_all_max

In [ ]:
print(f'precision_macro_min={precision_macro_min}, recall_macro_min={recall_macro_min},\
 score_macro_min={fscore_macro_min}, support_macro_min={support_macro_min}')

In [ ]:
print(f'precision_macro_max={precision_macro_max}, recall_macro_max={recall_macro_max},\
 score_macro_max={fscore_macro_max}, support_macro_max={support_macro_max}')

In [ ]:
print(f'precision_micro_min={precision_micro_min}, recall_micro_min={recall_micro_min},\
 score_micro_min={fscore_micro_min}, support_micro_min={support_micro_min}')

In [ ]:
print(f'precision_micro_max={precision_micro_max}, recall_micro_max={recall_micro_max},\
 score_micro_max={fscore_micro_max}, support_micro_max={support_micro_max}')

In [ ]:
#plot_confusion_matrix(clf, df3['Минимальный остаток'], df3['prophet'])
fig,ax = plt.subplots(num=None, figsize=(13, 10), dpi=80, facecolor='w', edgecolor='k')
confusion_matrix = pd.crosstab(df3_min['Минимальный остаток'], df3_min['prophet'],
                               rownames=['Минимальный остаток'], colnames=['prophet'])

sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='coolwarm')
ax.set_title('confusion_matrix for Минимальный остаток ')
plt.show()

In [ ]:
#plot_confusion_matrix(clf, df3['Минимальный остаток'], df3['prophet'])
fig,ax = plt.subplots(num=None, figsize=(16, 10), dpi=80, facecolor='w', edgecolor='k')
confusion_matrix = pd.crosstab(df3_max['Нормативный остаток'], df3_max['prophet'],
                               rownames=['Нормативный остаток'], colnames=['prophet'])

sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='coolwarm')
ax.set_title('confusion_matrix for Нормативный остаток')
plt.show()

In [ ]:
confusion_matrix

In [ ]:
# ЧТОЬ НЕ ЗАБЫТЬ:

## xgb_clf почитать о ней

## Где то здесь ошибка: 'KNeighborsClassifier_n_neighbors': [2, 3, 4, 5, 6, 7, 8]
## или ('KNeighborsClassifier', MultiOutputClassifier(KNeighborsClassifier()))

# clf = MultiOutputClassifier(KNeighborsClassifier()).fit(X, y)

# Так же посмотреть 

In [ ]:
# 1. Поработать с дисбалансом до testsplit (чтоб заработала стратификация)
# 2. Поигарть с метриками как в папке  MIN-MAX
# 3. Советуют попробовать так же MSE метрику (настойчиво советуют)
# 4. Добавить колонку  минимальный остаток, чтоб понимать по каким мин у нас проваливаются метрики
# 5. Может сравнение все же делать с данными без балансировки??

In [ ]:
##rfmSegmentation.to_csv('rfm-table_mark.csv', sep=';')

In [ ]:
score_all_min